1. First we read the json file that contains the tweets ( I have changed the file a bit to be able to read it)

In [1]:
import json

json_file = open('covidtrack_50K.json','r')
data = json.load(json_file)

2. Preprocessing

In [2]:
## Pre-Processing

import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import numpy as np
  
lemmatizer = WordNetLemmatizer() 

en_stops = set(stopwords.words('english'))

tweets_text = [None] * 50000

words_all = []


for i in range(50000):
    tweets_text[i] = data['data'][i]['text']
    
    #Convert text into lowercase
    tweets_text[i] = tweets_text[i].lower()
    
    #Remove non-alphabet characters
    tweets_text[i] = re.sub('[“?!""'':@#$%^&*()-_,…]', ' ', tweets_text[i])
    tweets_text[i] = re.sub("'s", ' ', tweets_text[i])
    
    #Break sentences into word
    tweets_text[i] = tweets_text[i].split()
    
    #Remove short word (length less than 3)
    tweet_words = [];
    for j in tweets_text[i]:
        if len(j)>2:
            tweet_words.append(j)
    tweets_text[i] = tweet_words
    
    #Lemmatization: converting a word to its base form
    for j in range(len(tweets_text[i])):        
        tweets_text[i][j] = lemmatizer.lemmatize(tweets_text[i][j])
     
    #Remove common English words
    tweet_words = [];
    for word in tweets_text[i]: 
        if word not in en_stops:
            tweet_words.append(word)
            words_all.append(word)
    tweets_text[i] = tweet_words
    
    #Remove short word (length less than 3)
    tweet_words = [];
    for j in tweets_text[i]:
        if len(j)>2:
            tweet_words.append(j)
    tweets_text[i] = tweet_words

print(len(words_all))

## remove repeated words and count each 
l = len(words_all)
c = 0
words = [];
counts = [];
list_words = words_all

while l > 0:
    word = list_words[0]
    
    words.append(word)
    
    while True:
        try:
            list_words.remove(word)
        except ValueError:
            break
    
    counts.append(l - len(list_words))

    l = len(list_words)
    #print(l)
        
words_array = np.array(words)    
 

print(len(words))

#Remove words that occur in less than 10 documents, and words that occur in more than 90% of the documents.
    
dictionary = []

for j in range(len(words)):
    #print(j)
    cnt = 0
    for i in range(50000):
        c = tweets_text[i].count(words[j])
        if c > 0:
            cnt = cnt + 1
    #print(cnt)
    if cnt>10:
        if cnt<45000:
            dictionary.append(words[j])
        
 
print(len(dictionary))

#calculate frequency of each word of the dictionary in each tweet

freq_vector =np.empty([50000, len(dictionary)], dtype=int)

for i in range(50000):
    #print(i)
    for j in range(len(dictionary)):
        c = tweets_text[i].count(dictionary[j])
        freq_vector[i][j] = c

        


590722
47488
4771


3. Topic modeling 

Applying LDA model on our frequency vector

In [3]:
##Topic modeling
import numpy as np
import lda
import gensim

model = lda.LDA(n_topics=20, n_iter=15, random_state=1)

model.fit(freq_vector)




INFO:lda:n_documents: 50000
INFO:lda:vocab_size: 4771
INFO:lda:n_words: 454276
INFO:lda:n_topics: 20
INFO:lda:n_iter: 15
INFO:lda:<0> log likelihood: -5352877
INFO:lda:<10> log likelihood: -3310850
INFO:lda:<14> log likelihood: -3193816


Examining the topics showing 7 words per topic

In [4]:
topic_word = model.topic_word_
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(dictionary)[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: http covid may end pandemic one lockdown
Topic 1: http death covid home new spread put
Topic 2: http family health year people pandemic many
Topic 3: http covid world amp died news china
Topic 4: http safe keep via ppe govt covid
Topic 5: http covid lockdown time week life breaking
Topic 6: http amp people place would trump country
Topic 7: death point play realcandaceo we’ve feeling watching
Topic 8: http trump latest president getting shutdown medical
Topic 9: http covid briefing medium drtedros realdonaldtrump china
Topic 10: http covid time lockdown life week lockdownextension
Topic 11: death case new breaking total number http
Topic 12: people continues crisis current yet suffering occupant
Topic 13: help people worker die research day video
Topic 14: http covid watch state home emergency japan
Topic 15: take know want question health news http
Topic 16: http covid iran spread pandemic say live
Topic 17: http pandemic new trump death crisis first
Topic 18: http covid pand